# Utility of Functions for Building NEU

---

# Loss Function(s)

In [1]:
def above_percentile(x, p): #assuming the input is flattened: (n,)

    samples = Kb.cast(Kb.shape(x)[0], Kb.floatx()) #batch size
    p =  (100. - p)/100.  #100% will return 0 elements, 0% will return all elements

    #selected samples
    values, indices = tf.math.top_k(x, samples)

    return values

def Robust_MSE_ES(p):
    def ES_p_loss(y_true, y_predicted):
        ses = Kb.pow(y_true-y_predicted,2)
        above = above_percentile(Kb.flatten(ses), p)
        return Kb.mean(above)
    return loss

In [3]:
# Tensorflow Version (Formulation with same arg-minima)
# @tf.function
def Robust_MSE(y_true, y_pred):
    # Compute Exponential Utility
    loss_out = tf.math.abs((y_true - y_pred))
    loss_out = tf.math.exp(robustness_parameter*loss_out)
    loss_out = tf.math.reduce_sum(loss_out)

    # Return Value
    return loss_out

In [ ]:
# Numpy Version (Full dual Version)
def Robust_MSE_numpy(y_true, y_pred):
    # Compute Exponential Utility
    loss_out = np.abs((y_true - y_pred))
    loss_out = np.exp(robustness_parameter*loss_out)
    loss_out = np.mean(loss_out)
    loss_out = np.log(loss_out)/robustness_parameter
    # Return Value
    return loss_out

---
# Base Models
## Regression
---

In [ ]:
# 1 hidden layer neural network
def def_model_Regression(trainx, trainy, Pre_Epochs_in, height, depth,learning_rate):
    #--------------------------------------------------#
    # Build Regular Arch.
    #--------------------------------------------------#
    
    #-###################-#
    # Define Model Input -#
    #-###################-#
    inputs_ffNN = tf.keras.Input(shape=(d,))
    
    x_ffNN = fullyConnected_Dense(height)(inputs_ffNN)
    #-##############################################################-#
    #### - - - (Reparameterization of) Feed-Forward Network - - - ####
    #-##############################################################-#
    for i in range(depth):
        #----------------------#
        # Choice of Activation #
        #----------------------#
        # ReLU Activation
        x_ffNN = tf.nn.relu(x_ffNN)
        # Sigmoid Activation
        #x_ffNN = tf.math.sigmoid(x_ffNN)
        # Leaky ReLU Activation
        #x_ffNN = Rescaled_Leaky_ReLU()(x_ffNN)
        
        #-------------#
        # Dense Layer #
        #-------------#
        x_ffNN = fullyConnected_Dense(height)(x_ffNN)
        
    
    
    #-###################-#
    # Define Output Layer #
    #-###################-#
    outputs_ffNN = fullyConnected_Dense(D)(x_ffNN)        
    
    # Define Model Output
    ffNN = tf.keras.Model(inputs_ffNN, outputs_ffNN)
    #--------------------------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)

    ffNN.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])
    
    ffNN.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=1)

    return ffNN

# Reconfigurations and NEU


In [2]:
projection_layer = tf.keras.layers.Lambda(lambda x: x[:, -D:])

In [6]:
# define and fit the base model
def get_base_model(trainx, trainy, Pre_Epochs_in, depth, height):
    # Define Model
    #----------------#
    # Initialize
    input_layer = tf.keras.Input(shape=[d+D])
    
    # Apply Reconfiguration Unit #
    #----------------------------#
    output_layer  = Reconfiguration_unit(height)(output_layer)
    
    if depth > 0:
        output_layer = Shift_Layers(d+D)(output_layer)
        output_layer  = Reconfiguration_unit(height)(output_layer)
        output_layer = Shift_Layers(d+D)(output_layer)
    
#     output_layer = projection_layer(output_layer)
    reconfiguration_basic = tf.keras.Model(inputs=[input_layer], outputs=[output_layer])
    
    # Compile Model
    #----------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.Adagrad(learning_rate=10**(-6))
    # Compile
    reconfiguration_basic.compile(loss = 'mae',#Robust_MSE,
                    optimizer = optimizer_on,
                    metrics = ['mse'])
    
    # Fit Model
    #----------------#
    reconfiguration_basic.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)
        
    # Return Output
    return reconfiguration_basic

#### Greedy Initialization of Subsequent Units
Build reconfiguration and pre-train using greedy approach.

In [ ]:
def add_reconfiguration_unit_greedily(model, trainx, trainy, Pre_Epochs_in, depth, height):

    # Dissasemble Network
    layers = [l for l in model.layers]

    # Define new reconfiguration unit to be added
    output_layer_new  = Reconfiguration_unit(d+D)(layers[len(layers)-1].output)

    if depth > 0:
        output_layer_new = Shift_Layers(d+D)(output_layer_new)
        output_layer_new  = Reconfiguration_unit(height)(output_layer_new)
        output_layer_new = Shift_Layers(d+D)(output_layer_new)

    for i in range(len(layers)):
        layers[i].trainable = False


    # build model
    new_model = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()


    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.Adagrad(learning_rate=10**(-6))
    # Compile Model
    new_model.compile(loss = 'mae',#Robust_MSE,
                    optimizer = optimizer_on,
                    metrics = ['mse'])

    # Fit Model
    #----------------#
    new_model.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)

    # Return Output
    return new_model

#### Train and Compile (entire) reconfiguration using greedy-initializations past from previous helper functions.
Train reconfiguration together (initialized by greedy) layer-wise initializations.

In [1]:
def build_reconfiguration(model_greedy_initialized, trainx, trainy, Full_Epochs_in, height):

    # Dissasemble Network
    layers = [l for l in model_greedy_initialized.layers]

    # Define new reconfiguration unit to be added
    output_layer_new  = Shift_Layers(d+D)(layers[len(layers)-2].output)
    output_layer_new  = Reconfiguration_unit(height)(output_layer_new)
    output_layer_new  = Shift_Layers(d+D)(output_layer_new)

    # Output Layer
#     output_layer_new = projection_layer(output_layer_new)

    for i in range(len(layers)):
        layers[i].trainable = True

    
    # Add Projection Layer (constructs full readout map)
    output_layer_new = projection_layer(output_layer_new)
    
    # build model
    reconfiguration = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()



    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-4))

    # Compile Model
    reconfiguration.compile(loss = Robust_MSE,
                    optimizer = optimizer_on,
                    metrics = ['mse','mae'])

    # Fit Model
    #----------------#
    reconfiguration.fit(trainx, trainy, epochs=Full_Epochs_in, verbose=1)

    # Return Output
    return reconfiguration

## NEU - Regerssion

In [ ]:
# 1 hidden layer neural network
def get_base_model_Regression(trainx, trainy, Pre_Epochs_in, height, depth, learning_rate):
    #--------------------------------------------------#
    # Build Regular Arch.
    #--------------------------------------------------#
    
    #-###################-#
    # Define Model Input -#
    #-###################-#
    inputs_ffNN = tf.keras.Input(shape=(d,))
    
    x_ffNN = fullyConnected_Dense(height)(inputs_ffNN)
    #-##############################################################-#
    #### - - - (Reparameterization of) Feed-Forward Network - - - ####
    #-##############################################################-#
    for i in range(depth):
        #----------------------#
        # Choice of Activation #
        #----------------------#
        # ReLU Activation
        x_ffNN = tf.nn.relu(x_ffNN)
        
        #-------------#
        # Dense Layer #
        #-------------#
        x_ffNN = fullyConnected_Dense(height)(x_ffNN)
        
    
    
    #-###################-#
    # Define Output Layer #
    #-###################-#
    x_ffNN = fullyConnected_Dense(D)(x_ffNN)     
    output_layer = tf.concat([inputs_ffNN, x_ffNN], axis=1)
    
    # Define Model Output
    ffNN = tf.keras.Model(inputs_ffNN, output_layer)
    #--------------------------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)

    ffNN.compile(optimizer=opt, loss=Robust_MSE, metrics=["mse", "mae", "mape"])
    
    ffNN.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=2)

    return ffNN